# Char level transformer for Machine translation
To do translation, we'll use sequence-to-sequence model: an encoder encode the input and decoder using both the encode and raw output to predict. 

This is similar to char-transformer for language modeling, but we will add an encoder, and in place of top-k output when we generate text we'll use top-1 beam search output.

<img src='images/transformer.png' width=400>
<a href="https://www.kaggle.com/hungnm/englishvietnamese-translation">English-Vietnamese dataset</a>

In [5]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
class CharDataset(Dataset):
    '''
    Dataset is a iterable that returns input and target sentence. It adds <sos> at the begining, and <eos> at the end, 
    and filling in <pad> if sentence length is less than pre-defined value.
    '''
    def __init__(self, x, y, sequence_len):
        # data in the type of pairs of sentence

        data = ''.join(x+y)
        chars = ['<pad>'] +['<sos>'] + ['<eos>'] + sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)

        print('data has %d characters, %d unique chars, %d sentences.' % (data_size, len(chars), len(x)))
        print('%d sentence length' % (sequence_len))
        
        self.x, self.y = x, y
        self.ch2i = {ch:i for i,ch in enumerate(chars)}
        self.i2ch = {i:ch for i,ch in enumerate(chars)}
        self.vocab_size = vocab_size
        self.sequence_len = sequence_len
    
    def __len__(self):
        return len(self.x) # len x = y
    
    def __getitem__(self, idx):
        indx = self.padding([self.ch2i[ch] for ch in self.x[idx]] + [self.ch2i['<eos>']])
        indy = [self.ch2i['<sos>']] + self.padding([self.ch2i[ch] for ch in self.y[idx]] + [self.ch2i['<eos>']])

        x = torch.tensor(indx, dtype=torch.long)
        y = torch.tensor(indy, dtype=torch.long)
                                                                                                                               
        return x,y
                                                                                                                               
    def padding(self, string):
        if len(string)<self.sequence_len:
            string =  string + [0]*(self.sequence_len - len(string))
        else:
            string = string[:self.sequence_len -1] + [self.ch2i['<eos>']]
                   
        return string
                                                                                                                               


In [18]:
# walk through data folder
import os
text = {}
for (dirpath, dirnames, filenames) in os.walk("data"):
    for name in filenames:
        path = os.path.join(dirpath, name)
        with open(path, encoding='utf-8') as f:
            text[name] = f.read().lower().split('\n')

x,y = text['vi_sents'], text['en_sents']

# print some samples
for i in range(-1,-3,-1):
    print(x[i],'|',y[i])

bạn đã ở đám cưới của tôi | you were at my wedding
tại sao bạn không quan tâm? | why aren't you interested?


In [22]:
nine_nine_percentile = int(np.percentile([len(sen) for sen in x],99))
dataset = CharDataset(x, y, sequence_len=nine_nine_percentile)

print('sample tensors ', next(iter(dataset)))
print("vocab: ", dataset.ch2i)

data has 17367685 characters, 151 unique chars, 254090 sentences.
81 sentence length
sample tensors  (tensor([ 51,  36,  41,   3,  49,  48,  36,   3,  39,  71,  41,  34,   3,  82,
        103,  47,   3,  41,  34,  86, 122,  36,   3,  44,  48,  65,  47,   3,
         45,  60,  30,   3,  47,  45,  42,  41,  34,   3,  47, 127,   3,  30,
         35, 118,  36,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]), tensor([ 1, 43, 39, 32, 28, 46, 32,  3, 43, 48, 47,  3, 47, 35, 32,  3, 31, 48,
        46, 47, 43, 28, 41,  3, 36, 41,  3, 47, 35, 32,  3, 29, 45, 42, 42, 40,
         3, 30, 39, 42, 46, 32, 47,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0]))
vocab:  {'<pad>': 0, '<sos>': 1, '<eos>': 2, ' ': 3, '!': 4, '"': 5, '$':

# Building model

In [9]:
from model.encode_decode_transformer import Transformer, TransformerConfig
from utils.trainer import Trainer, TrainerConfig
tconfig = TrainerConfig(max_epochs=2, batch_size=16, learning_rate=6e-4, grad_norm_clip=1.0, device='cuda',
                       lr_decay=True, warmup_tokens=5000, ckpt_n_print_iter=500, ckpt_path='checkpoint/transformer_vn_en')

mconfig = TransformerConfig(vocab_size=dataset.vocab_size, sequence_len=dataset.sequence_len, embed_dim=256,
                           n_block=8, n_head=8, causal=None, device=tconfig.device)

In [12]:
model = Transformer(mconfig)
trainer = Trainer(model, dataset, tconfig, test_dataset=None, collate=None)

09/27/2021 20:00:08 - INFO - model.encode_decode_transformer -   number of parameters: 1.482982e+07


In [ ]:
# load pre-trained weights
from utils.utils import pickle
model.load_state_dict(pickle(tconfig.ckpt_path)) # load

In [23]:
trainer.train()

epoch: 1 | train loss: 0.32667  | lr: 1.920000e-06:   0%|                                    | 0/15880 [00:00<?, ?it/s]

[]


epoch: 1 | train loss: 0.94756  | lr: 5.997830e-04:   3%|▊                         | 500/15880 [01:23<38:22,  6.68it/s]

["mất bao lâu để bay từ tokyo đến los angeles?   <sos>you don't you you you you you you?<eos>", 'đó là một cái túi mà tôi bị mất trong phòng ngày hôm qua.   <sos>tom and to to to the the the the the the the the the the the t the the the <eos>', 'mary đang mặc một chiếc cà vạt   <sos>tom want to to to to the the the the the the ther<eos>', 'tôi muốn một cuốn sách   <sos>tom want to to the to the the this<eos>']


epoch: 1 | train loss: 0.86248  | lr: 5.971089e-04:   6%|█▌                       | 1000/15880 [02:47<37:53,  6.55it/s]

...

epoch: 2 | train loss: 0.39251  | lr: 5.980486e-04:  94%|██████████████████████▋ | 15002/15880 [38:26<10:57,  1.33it/s]

['mất bao lâu để bay từ tokyo đến los angeles?   <sos>how long does tom start from the station is listening?<eos>', 'đó là một cái túi mà tôi bị mất trong phòng ngày hôm qua.   <sos>it was a little time i was in this room.<eos>', 'mary đang mặc một chiếc cà vạt   <sos>mary is businessing and the children.<eos>', "tôi muốn một cuốn sách   <sos>i'd like some children.<eos>"]


epoch: 2 | train loss: 0.38302  | lr: 5.999738e-04:  98%|███████████████████████▍| 15502/15880 [39:42<04:32,  1.39it/s]

['mất bao lâu để bay từ tokyo đến los angeles?   <sos>how much longer does the next time?<eos>', "đó là một cái túi mà tôi bị mất trong phòng ngày hôm qua.   <sos>that's a very little time i've been in this room.<eos>", 'mary đang mặc một chiếc cà vạt   <sos>mary was listening for a few house<eos>', "tôi muốn một cuốn sách   <sos>i'd like some books.<eos>"]


epoch: 2 | train loss: 0.34331  | lr: 5.994036e-04: 100%|████████████████████████| 15880/15880 [40:38<00:00,  6.51it/s]


In [39]:
samples = ["thời tiết hôm nay thật đẹp!", 
          "xin chào", 
          "bạn đã ăn sáng chưa?"]
result = model.generate_output(samples, dataset, top_k=3)
print(result)

thời tiết hôm nay thật đẹp!   <sos>what is the weather for the mountain!<eos>
xin chào   <sos>please come.<eos>
bạn đã ăn sáng chưa?   <sos>have you eaten the morning?<eos>

